# Segmenting and Clustering Neighborhoods in Toronto

## Table of contents
* [Part 1](#section1)
* [Part 2](#section2)
* [Part 3](#section3)

## Part 1 - Scraping the data
<a id="#section1"></a>

In [1]:
#Import
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup # this module helps in web scrapping.

In [60]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(URL).text
soup = BeautifulSoup(data,"html5lib")

#find all html tables in the web page
tables = soup.find_all('table') # in html table is represented by the tag <table>
print(f"There are {len(tables)} tables on this website.")

# target table is table 0
targetTable = tables[0]
#print(tables[0].prettify())

There are 3 tables on this website.


In [68]:
# Creating Dataframe from scraped data
data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in targetTable.tbody.find_all("tr"):
    fields = row.find_all("td")
    
    for field in fields:
        postalCode = field.find("b").text
        if field.find("span").text == "Not assigned":
            continue
        borough = field.find("span").text.split("(")[0]
        neighborhood = field.find("span").text.split("(")[1]
        neighborhood = neighborhood.split(")")[0]
        
        # if neighborhoods are separeted by "/", replace by delimiter ","
        if "/" in neighborhood:
            neighborhood = neighborhood.replace("/", ",")
        
        data = data.append({"PostalCode":postalCode,
                            "Borough":borough,
                            "Neighborhood":neighborhood}, ignore_index=True)

In [70]:
data.shape

(103, 3)

# Part 2 - Using API to get Latitude and Longitude
<a id="#section1"></a>

In [77]:
geodata = pd.read_csv("Geospatial_Coordinates.csv")
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [78]:
data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [82]:
lat = []
long = []

for postalCode in data['PostalCode']:
    lat.append(geodata[geodata['Postal Code'] == postalCode]["Latitude"].values[0])
    long.append(geodata[geodata['Postal Code'] == postalCode]["Longitude"].values[0])
    

In [85]:
data['Latitude'] = lat
data['Longitude'] = long

In [86]:
data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
